# CS540 Pytorch Tutorial - Variable

Variable
--------

``autograd.Variable`` is the central class of the package. It wraps a
Tensor, and supports nearly all of operations defined on it. Once you
finish your computation you can call ``.backward()`` and have all the
gradients computed automatically.

You can access the raw tensor through the ``.data`` attribute, while the
gradient w.r.t. this variable is accumulated into ``.grad``.

.. figure:: /_static/img/Variable.png
   :alt: Variable

   Variable

There’s one more class which is very important for autograd
implementation - a ``Function``.

``Variable`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each variable has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Variable`` (except for Variables created by the user - their
``grad_fn is None``).

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Variable``. If ``Variable`` is a scalar (i.e. it holds a one element
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``grad_output``
argument that is a tensor of matching shape.


In [228]:
import torch
from torch.autograd import Variable

Create a varible:

In [229]:
x = Variable(torch.ones(1,1), requires_grad=True)
print(x)

Variable containing:
 1
[torch.FloatTensor of size 1x1]



In [230]:
y = 2 * x + 3
print(y)

Variable containing:
 5
[torch.FloatTensor of size 1x1]



In [231]:
z = y * y * 3

print(z)

Variable containing:
 75
[torch.FloatTensor of size 1x1]



To reduce memory usage, during the .backward() call, all the intermediary results are deleted when they are not needed anymore. Hence if you try to call .backward() again, the intermediary results don’t exist and the backward pass cannot be performed (and you get the error you see).
You can call .backward(retain_graph=True) to make a backward pass that will not delete intermediary results, and so you will be able to call .backward() again. All but the last call to backward should have the retain_graph=True option.
Also, the gradient is accumulated, it needs to be cleared each time if you do not want accumulated gradients. 

In the example above, ```dz/dx = dz/dy * dy/dx = 6y * 2 = 12y```, since ```y = 5```, ```dz/dx = 60```

In [232]:
z.backward()
#if x.grad is not None:
#    x.grad.data.zero_()
#z.backward(retain_graph=True)
print(x.grad)

Variable containing:
 60
[torch.FloatTensor of size 1x1]



For multiple dimensions

In [234]:
x = Variable(torch.ones(2,2), requires_grad=True)
y = 2 * x + 3
z = y * y * 3
print(z)

Variable containing:
 75  75
 75  75
[torch.FloatTensor of size 2x2]



In [241]:
z.backward(torch.ones(2,2), retain_graph=True)
print(x.grad)
x.grad.data.zero_()
z.backward(torch.Tensor([[0,1],[1,0]]), retain_graph=True)
print(x.grad)
x.grad.data.zero_()
z.backward(torch.Tensor([0,1]), retain_graph=True)
print(x.grad)
x.grad.data.zero_()
z.backward(torch.Tensor([[0],[1]]), retain_graph=True)
print(x.grad)
x.grad.data.zero_()

Variable containing:
 60  60
 60  60
[torch.FloatTensor of size 2x2]

Variable containing:
  0  60
 60   0
[torch.FloatTensor of size 2x2]

Variable containing:
  0  60
  0  60
[torch.FloatTensor of size 2x2]

Variable containing:
  0   0
 60  60
[torch.FloatTensor of size 2x2]




 0  0
 0  0
[torch.FloatTensor of size 2x2]